# Authenticate

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()


Use https://ucr.maps.arcgis.com/ to sign in with ucr credentials so you have access to arcgis pro and api development permissions.

Then go to content > new item > developer credential > continue > then type in redirect url = urn:ietf:wg:oauth:2.0:oob and url you can use https://localhost. This will create the client_id, which you should paste into the following cell, and then when you run it it should send you a password link. Type the password link into the box.

In [20]:
from arcgis.gis import GIS
import arcgis


from arcgis.map import Map
print(arcgis.__version__)


# Authenticate GIS.
my_client_id = "Sry1p3Nb1sg3fciJ"
gis = GIS("https://ucr.maps.arcgis.com", client_id=my_client_id)

2.4.0
Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucr.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Sry1p3Nb1sg3fciJ&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=2lodEEA9itUKsBNsEY2cQ9AESx0xtE&allow_verification=false


Enter code obtained on signing in using SAML:  ········


# Global Data

In [21]:
    crimeTypes = ['DRUG ABUSE VIOLATIONS', 'OTHER: DUI', 'ASSAULT: Other Assaults',
           'OTHER: All Other Offenses', 'WEAPONS: Carrying, Possessing, etc.',
           'Other Assaults - Not Aggravated',
           'ASSAULT: Knife or cutting instrument', 'THEFT: Shoplifting',
           'THEFT: Theft from motor vehicle', 'ASSAULT: Firearm',
           'SEX CRIMES: Sex Offenses', 'OTHER: Stolen Property',
           'FRAUD: Other Fraud', 'FRAUD: Forgery & Counterfeiting',
           'VANDALISM: Vandalism', 'MOTOR VEH. THEFT: Other vehicles',
           'BURGLARY: Forcible entry', 'DRUG ABUSE VIOLATIONS: Drunkenness',
           'FRAUD: Embezzlement', 'THEFT: All other larceny',
           'MOTOR VEH. THEFT: Autos', 'MOTOR VEH. THEFT: Trucks and buses',
           'ASSAULT: Strong-Arm', 'ASSAULT: Other dangerous weapon',
           'ROBBERY: Firearm', 'ROBBERY: Strong-Arm',
           'THEFT: Theft of motor vehicle parts or accessories',
           'THEFT: Theft of bicycles', 'BURGLARY: Unlawful entry - no force',
           'SEX CRIMES: Sexual Assault', 'OTHER: Offenses Against Family',
           'THEFT: Theft from buildings',
           'THEFT: Theft from coin-operated machine or device',
           'DISTURBING THE PEACE: Disorderly Conduct',
           'THEFT: Pocket picking', 'BURGLARY: Attempted forcible entry',
           'ROBBERY: Knife or cutting instrument',
           'OTHER: Curfew and Loitering',
           'DRUG ABUSE VIOLATIONS: Liquor Laws',
           'HOMICIDE: Murder & non-negligent manslaughter', 'GAMBLING',
           'ROBBERY: Other dangerous weapon', 'SEX CRIMES: Prostitution/Vice',
           'THEFT: Purse snatching', 'DRIVING UNDER THE INFLUENCE',
           'DESTRUCTION / DAMAGE / VANDALISM OF PROPERTY', 'SIMPLE ASSAULT',
           'ALL OTHER OFFENSES', 'WEAPON LAW VIOLATIONS',
           'FAMILY OFFENSES, NONVIOLENT', 'BURGLARY/BREAKING AND ENTERING',
           'AGGRAVATED ASSAULT',
           'THEFT OF MOTOR VEHICLE PARTS OR ACCESSORIES', 'ALL OTHER LARCENY',
           'CREDITCARD / AUTOMATED TELLER MACHINE FRAUD', 'SHOPLIFTING',
           'DISORDERLY CONDUCT', 'MOTOR VEHICLE THEFT',
           'KIDNAPPING / ABDUCTION', 'STOLEN PROPERTY OFFENSES',
           'THEFT FROM BUILDING', 'FALSEPRETENSES / SWINDLE / CONFIDENCEGAME',
           'TRESPASS OF REAL PROPERTY', 'ROBBERY', 'IDENTITYTHEFT',
           'DRUG EQUIPMENT VIOLATIONS', 'IMPERSONATION',
           'DRUG NARCOTIC VIOLATIONS', 'THEFT FROM MOTOR VEHICLE',
           'SEX CRIMES', 'POCKET-PICKING', 'EMBEZZLEMENT',
           'HACKING / COMPUTERINVASION', 'COUNTERFEITING / FORGERY',
           'LIQUOR LAW VIOLATIONS', 'INTIMIDATION',
           'PORNOGRAPHY / OBSCENEMATERIAL',
           'CURFEW / LOITERING / VAGRANCY VIOLATIONS',
           'EXTORTION / BLACKMAIL', 'PROSTITUTION', 'WELFAREFRAUD',
           'MURDER AND NON-NEGLIGENT MANSLAUGHTER', 'ARSON',
           'THEFT FROM COIN-OPERATED MACHINE OR DEVICE', 'ANIMAL CRUELTY',
           'PURSE-SNATCHING', 'HUMANTRAFFICKING ,SERVITUDE INVOLUNTARY',
           'WIREFRAUD', 'OPERATING / PROMOTING / ASSISTING GAMBLING']

# Create Police Report Dataframe

In [22]:


import pandas as pd

def create_report_dataframe_riverside():
    df = spark.read.csv("Riverside_Crime_Reports.csv", header=True, inferSchema=True).select("rd", "blockAddress", "crimeType")
    return df

def create_report_dataframe_la():
    df = spark.read.csv("LA_Crime_Data.csv", header=True, inferSchema=True).select('Rpt Dist No', 'LOCATION', 'Crm Cd Desc', 'LAT', 'LON')
    return df
    
def create_report_dataframe(df_riverside, df_la):
    df_la = (df_la.withColumnRenamed('Rpt Dist No', 'rd')
             .withColumnRenamed('LOCATION', 'blockAddress')
             .withColumnRenamed('Crm Cd Desc', 'crimeType')
             .select('rd', 'blockAddress', 'crimeType'))
    df_merged = df_riverside.unionByName(df_la)
    return df_merged

    
    
crime_df_riverside = create_report_dataframe_riverside()
crime_df_la = create_report_dataframe_la()
crime_df = create_report_dataframe(crime_df_riverside, crime_df_la)

display(crime_df_riverside.head())
display(crime_df_la.head())
display(crime_df.head())

Row(rd='J14', blockAddress='Sagittarius Dr / Boundary Ln', crimeType='DRUG ABUSE VIOLATIONS')

Row(Rpt Dist No=784, LOCATION='1900 S  LONGWOOD                     AV', Crm Cd Desc='VEHICLE - STOLEN', LAT=34.0375, LON=-118.3506)

Row(rd='J14', blockAddress='Sagittarius Dr / Boundary Ln', crimeType='DRUG ABUSE VIOLATIONS')

# Create shape dataframe

Riverside points initially need to be geocoded since the crimes are not given x,y coordinates, but rather blockAddresses. Geocoding converts the block address to an x,y point. Lots of these API calls need to be made and this takes time, so ArcGIS will de-authenticate before the process finishes. CreatePointDataframe has been implemented so that batches of geocodings are saved to a file, and when runtime errors such as Response Not Subscriptable occur the cell can just be re-run to continue geocoding until everything has been geocoded.

In [23]:
from arcgis.geometry import Point
from arcgis.geocoding import geocode

def geocode_block_address(block_address: str):
    if not block_address or not isinstance(block_address, str):  # Check for None or invalid input
        return None  # Return None for invalid addresses

    try:
        # Perform geocoding
        results = geocode(block_address, ", Riverside, CA")
        if len(results) == 0:
            return None
        best_result = results[0]
        print(best_result)
        best_result['location'].update({"spatialReference": {"wkid": 4326}})
        return Point(best_result['location'])
    except Exception as e:
    # Handle geocoding errors. Typically caused by losing GIS Api Authentification, which times out every ~30 mins
        print(f"Error geocoding address '{block_address}': {e}")
        raise RuntimeError("Probably need to restart execution of this cell after re-authenticating arcgis.")
        return None



In [24]:

import pandas as pd
from arcgis.features import FeatureSet

def create_shape_dataframe_riverside():
    # Access a web map of riverside reporting districts (RDs)
    r_item = gis.content.get("4855e57db5d3430cb94ddd32688ab7b7")
    r_feature_collection = r_item.layers[0]
    # Update Esri Requirement for this layer
    r_feature_collection.properties.layerDefinition.htmlPopupType = 'esriServerHTMLPopupTypeAsHTMLText'
    
    #Convert featurecollection to featureset to get actual features, then covert to spatial dataframe
    r_fset = r_feature_collection.query()
    r_fset_df = r_fset.sdf
    return r_fset_df[['NAME', 'SHAPE']].rename(columns={"NAME": "rd"})

def create_shape_dataframe_la():
    # Access a web map of riverside reporting districts (RDs)
    la_item = gis.content.get("721a305c2aac4fc08a816787e8740a98")
    la_feature_collection = la_item.layers[0]
    
    #Convert featurecollection to featureset to get actual features, then covert to spatial dataframe
    la_fset = la_feature_collection.query()
    la_fset_df = la_fset.sdf
    return la_fset_df[['NAME', 'SHAPE']].rename(columns={"NAME": "rd"})

def create_shape_dataframe():
    rd_geometry = create_shape_dataframe_riverside()
    la_rd_geometry = create_shape_dataframe_la()
    return pd.concat([rd_geometry, la_rd_geometry], ignore_index=True)
    



import pickle

# For Riverside data, all block addresses need to be geocoded since x,y locations are not given.
# Results from geocoding are saved to a file in batches since runtime errors are common due to the 
# magnitude of geocode api calls made. Re-run this cell as many times as needed until 
def create_point_dataframe_riverside(crime_df_riverside, file_path = ''):
    
    crime_df_pd = crime_df_riverside.toPandas()
    invalid_points = []
    
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            geocoded_points = pickle.load(file)
        print("Loaded geocoded points from file.")
    else:
        print("Cannot find file with pre-generated geocodings. Geocoding all points. This may take a while and require re-authenticating arcgis. Consider uploading the file containing the correct geocoded locations.")
        geocoded_points = {}
    
    
    # Go through every block address. Remembering invalid points for error checking with response not available
    for i, blk_address in enumerate(crime_df_pd['blockAddress']):
        if i % 500 == 0:
            with open(file_path, 'wb') as file:
                pickle.dump(geocoded_points, file)
    
        if blk_address in geocoded_points:
            continue
        else:
            geocoded_point = geocode_block_address(blk_address)
            geocoded_points[blk_address] = geocoded_point
        if geocoded_point is None:
            invalid_points.append(i)
    
    with open(file_path, 'wb') as file:
        pickle.dump(geocoded_points, file)

    
    blockAddress_geometry = pd.DataFrame({'blockAddress': crime_df_riverside.toPandas()['blockAddress'].unique()})
    blockAddress_geometry['SHAPE'] = blockAddress_geometry['blockAddress'].apply(lambda blk_address: geocoded_points[blk_address])
    return blockAddress_geometry


def create_point_dataframe_la(crime_df_la, file_path = ''):
    crime_df_la = crime_df_la.select('LOCATION', 'LAT', 'LON').toPandas()
    crime_df_la['SHAPE'] = crime_df_la.apply(lambda row: arcgis.geometry.Point({'x': row['LAT'], 'y': row['LON'], 'spatialReference': {'wkid': 4326}}),axis=1)
    crime_df_la = crime_df_la.drop(columns=['LAT', 'LON']).rename(columns={"LOCATION": "blockAddress"})
    return crime_df_la


def create_point_dataframe():
    blockAddress_geometry = create_point_dataframe_riverside(crime_df_riverside, "geocoded_points.pkl")
    location_geometry = create_point_dataframe_la(crime_df_la)
    return pd.concat([blockAddress_geometry, location_geometry], ignore_index=True)

shape_df = create_shape_dataframe()
point_df = create_point_dataframe()

display(shape_df)
display(point_df)



Loaded geocoded points from file.


,rd,SHAPE
0,K03,"{""rings"": [[[-13059322, 4013797], [-13059306, ..."
1,K05,"{""rings"": [[[-13061268, 4013817], [-13061269, ..."
2,J20,"{""rings"": [[[-13073478, 4013648], [-13074932, ..."
3,J07,"{""rings"": [[[-13076145, 4014008], [-13076824, ..."
4,I15,"{""rings"": [[[-13068909, 4015593], [-13067078, ..."
...,...,...
1264,0406,"{""rings"": [[[-13155515.5846904, 4042485.918871..."
1265,0405,"{""rings"": [[[-13155629.2003684, 4043645.800824..."
1266,1701,"{""rings"": [[[-13199168.9852206, 4068530.010192..."
1267,1251,"{""rings"": [[[-13172045.4832266, 4025869.175404..."


,blockAddress,SHAPE
0,Sagittarius Dr / Boundary Ln,"{'x': -117.456645469102, 'y': 33.895308997468,..."
1,10300 BLOCK CAMPBELL AVE,"{'x': -117.478330717128, 'y': 33.936321735676,..."
2,5800 BLOCK CHALLEN AVE,"{'x': -117.46579873813, 'y': 33.941310611459, ..."
3,E Alessandro Blvd / Mission Grove Pkwy,"{'x': -117.330934916792, 'y': 33.917409046214,..."
4,2900 BLOCK MARY ST,"{'x': -117.38877287487, 'y': 33.936060077028, ..."
...,...,...
1004402,4000 W 23RD ST,"{'x': 34.0362, 'y': -118.3284, 'spatialReferen..."
1004403,1300 W SUNSET BL,"{'x': 34.0685, 'y': -118.246, 'spatialReferenc..."
1004404,1700 ALBION ST,"{'x': 34.0675, 'y': -118.224, 'spatialReferenc..."
1004405,FLOWER ST,"{'x': 34.0215, 'y': -118.2868, 'spatialReferen..."


# Query Police Report Dataframe

In [25]:

from pyspark.sql.functions import col

# query the crime_df by crimetype. Default is all crime types.
def query_reports(df, crimeType: str = "", condition_code: int = 0, condition: int = 0, points: bool = False):
    # Filter rows where crimeType is 'ROBBERY' and group by rpc column, counting the occurrences
    query_present = False
    query_string = ''

    # Handle incorrect input for query_string. This will default to all crime types if there is an error
    if crimeType != "":
        if crimeType in crimeTypes:
            print("Querying... ")
            query_string = crimeType
            query_present = True
        else:
            print("Warning: Invalid Crime Type")
            query_string = 'Invalid Query: Returning All Crimes'
            query_present = False
    else:
        query_string = 'All Crimes'
        
    # Perform the query for crimeType
    if query_present:
        query_result = (df.select("rd", "blockAddress", "crimeType")
                      .filter(df.crimeType == crimeType))
    else:
        query_result = df

    # Get counts by blockAddress and by reporting distrits
    queried_df_blockAddress = (query_result.select("blockAddress")
                         .groupby("blockAddress")
                         .count())
    
    queried_df_rd = (query_result.select("rd", "crimeType")
                      .groupBy("rd")
                      .count())
    
    # Return the dataframes as pandas dataframes
    print("Querying Finished")
    queried_df_blockAddress = queried_df_blockAddress.toPandas()
    queried_df_rd = queried_df_rd.toPandas()
    
    return queried_df_blockAddress, queried_df_rd, query_string
    
queried_df_blockAddress, queried_df_rd, query_string = query_reports(crime_df)
display(queried_df_blockAddress)
display(queried_df_rd)

Querying Finished


,blockAddress,count
0,3500 BLOCK POLK ST,132
1,1000 BLOCK SPRUCE ST,188
2,5th St / Pine St,1
3,Lila St/jefferson St,1
4,4500 BLOCK ARLINGTON AVE,35
...,...,...
80474,92ND AV,1
80475,LMU DR,1
80476,2300 S BEVERLY DR,1
80477,7100 MCLAREN AV,1


,rd,count
0,B05,362
1,Z21,25
2,E02,983
3,G12,367
4,17A,288
...,...,...
1419,1100,1
1420,995,1
1421,1046,2
1422,709,1


# Merge Dataframes

In [26]:
# Merge the queried dataframe with the point and shape dataframes to give the crimes geometries
def merge_dataframes(queried_df_pd, geometry_df_pd, col):
    merged_df = pd.merge(queried_df_pd, geometry_df_pd, on=col, how="outer")
    merged_df["count"] = merged_df["count"].fillna(0).astype(int)
    return merged_df

df = merge_dataframes(queried_df_rd, shape_df, "rd")
df2 = merge_dataframes(queried_df_blockAddress, point_df, "blockAddress")
display(df)
display(df2)


,rd,count,SHAPE
0,B05,362,"{""rings"": [[[-13058817, 4026512], [-13058816, ..."
1,Z21,25,None
2,E02,983,"{""rings"": [[[-13068347, 4025106], [-13068347, ..."
3,G12,367,"{""rings"": [[[-13070011, 4019232], [-13070772, ..."
4,17A,288,None
...,...,...,...
1894,0404,0,"{""rings"": [[[-13155863.5509396, 4043737.643512..."
1895,0403,0,"{""rings"": [[[-13156937.2040512, 4042767.872046..."
1896,0406,0,"{""rings"": [[[-13155515.5846904, 4042485.918871..."
1897,0405,0,"{""rings"": [[[-13155629.2003684, 4043645.800824..."


,blockAddress,count,SHAPE
0,3500 BLOCK POLK ST,132,"{'x': -117.466222838411, 'y': 33.904545023372,..."
1,1000 BLOCK SPRUCE ST,188,"{'x': -117.334138494748, 'y': 33.990174826867,..."
2,5th St / Pine St,1,"{'x': -117.384641418932, 'y': 33.982660960342,..."
3,Lila St/jefferson St,1,"{'x': -117.392267626779, 'y': 33.903144477033,..."
4,4500 BLOCK ARLINGTON AVE,35,"{'x': -117.402927090635, 'y': 33.946190570161,..."
...,...,...,...
1004402,92ND AV,1,"{'x': 33.9527, 'y': -118.309, 'spatialReferenc..."
1004403,LMU DR,1,"{'x': 33.9655, 'y': -118.4247, 'spatialReferen..."
1004404,2300 S BEVERLY DR,1,"{'x': 34.0425, 'y': -118.3949, 'spatialReferen..."
1004405,7100 MCLAREN AV,1,"{'x': 34.1992, 'y': -118.6245, 'spatialReferen..."


# Graph Result


In [27]:
from arcgis.map.popups import FieldInfo, PopupExpressionInfo
import ipywidgets as widgets
from arcgis.features import FeatureCollection
from arcgis.map import symbols



def create_map():
    # Create the map using our featureCollection
    my_map = Map()
    my_map.basemap.basemap =  "arcgis-streets"
        # Update starting field of view, and display the map
    my_map.extent = {'spatialReference': {'wkid': 102100},
                     'xmin': -13087454.153546248, 'ymin': 4009445.945260928,
                     'xmax': -13044725.85473489, 'ymax': 4032377.0537464498}
    return my_map
    

    
def add_layer(my_map, df, df2):
    # setup map data
    fset = FeatureSet.from_dataframe(df)
    feature_collection = FeatureCollection.from_featureset(fset)
    # fset2 = FeatureSet.from_dataframe(df2)

    # setup map popup info
    expression_infos = [
        PopupExpressionInfo(
            title="Name",
            expression="return $feature.NAME;"
        )
    ]
    
    field_infos = [
        FieldInfo(
            fieldName="rd",
            label="Reporting District",
            visible=True
        ),
        FieldInfo(
            fieldName="Count",
            label="Number of Police Reports",
            visible=True
        )
    ]

    # Add new layer and update popup info
    my_map.content.remove_all()
    my_map.content.add(feature_collection)
    my_map.content.popup(0).edit(expression_infos=expression_infos, field_infos=field_infos)

    # Use smartmapping to make a class_Breaks_renderer with classes and color based on the count field
    my_map.content.renderer(0).smart_mapping().class_breaks_renderer(break_type="color", field="count", num_classes=30)
    
    
    # my_map.content.add(feature_collection2, options={"visibility": False})
    
    # # Update the symbol size in a way that ArcGIS will recognize the renderer has been updated
    # renderer = my_map.content.renderer(1).renderer
    # renderer.symbol.size = 2
    # my_map.content.renderer(1).renderer = renderer
    
    



def create_combobox(my_map, crime_df, shape_df):
    # Create text/dropdown box
    combobox = widgets.Combobox(
        placeholder="Type or Select Crime Type",
        options=crimeTypes,
        description="Crime Type",
    )
    # Submit on enter/tab clicked only
    combobox.continuous_update = False
    
    #Create a place for output
    output = widgets.Output()
    
    # Handle finishing
    def on_combobox_submit(change):
        with output:
            output.clear_output()
            my_map.content.remove_all()
            print("working")


            queried_df_blockAddress, queried_df_rd, query_string = query_reports(crime_df, combobox.value)
            df = merge_dataframes(queried_df_rd, shape_df, "rd")
            
            add_layer(my_map, df, df2)

            output.clear_output()
            print(f"Displaying: {query_string}")

    
    # Attach the handler
    combobox.observe(on_combobox_submit, names="value")
    
    # Display widgets
    return combobox, output
    
# Toggle the visibility of the layer that displays points
def create_pointsbox(my_map):
    checkbox = widgets.Checkbox(
    value=False,
    description='Display Points'
    )

    def on_checkbox_submit(change):
        if change['new']:
            my_map.content.layer_visibility(1).visibility= True
        else:
            my_map.content.layer_visibility(1).visibility= False
        my_map.content.update_layer(1)
        
        
    checkbox.observe(on_checkbox_submit, names="value")
    return checkbox

# Toggle the opacity. This updates the opacity for all class breaks within the renderer
def create_opacitybox(my_map):
    checkbox = widgets.Checkbox(
    value=False, 
    description='is_opaque'
    )

    def on_checkbox_submit(change):
        if change['new']:
            renderer = my_map.content.renderer(0).renderer
            for cls in renderer.class_break_infos:
                cls.symbol.color[3] = 100  
            my_map.content.renderer(0).renderer = renderer
            
        else:
            renderer = my_map.content.renderer(0).renderer
            for cls in renderer.class_break_infos:
                cls.symbol.color[3] = 255  
            my_map.content.renderer(0).renderer = renderer

        
        
    checkbox.observe(on_checkbox_submit, names="value")
    return checkbox

# Select a threshold within text field. This updates the renderer to have two classbreaks,
# one for below the threshold and one for above.
def create_numberbox(my_map):
    # Create a number input box
    number_box = widgets.Text(
        placeholder="None",
        description="Threshold:",
    )
    # Submit on enter/tab clicked only
    number_box.continuous_update = False

    
    def is_int(value):
        try:
            int(value)
            return True
        except ValueError:
            return False
    
    def on_number_submit(change):
        val = int(number_box.value)
        my_map.content.base_renderer = my_map.content.renderer(0).renderer
        cbi = arcgis.map.renderers.ClassBreakInfo(class_max_value=val, symbol=symbols.SimpleFillSymbolEsriSFS(color=[0, 255, 0, 100], outline=symbols.SimpleLineSymbolEsriSLS(color=[153, 153, 153, 64], marker=None, style='esriSLSSolid', type='esriSLS', width=0.375), style='esriSFSSolid', type='esriSFS'))
        cbi2 = arcgis.map.renderers.ClassBreakInfo(class_min_value=val, class_max_value=5000, symbol=symbols.SimpleFillSymbolEsriSFS(color=[255, 0, 0, 100], outline=symbols.SimpleLineSymbolEsriSLS(color=[153, 153, 153, 64], marker=None, style='esriSLSSolid', type='esriSLS', width=0.375), style='esriSFSSolid', type='esriSFS'))    
        my_map.content.renderer(0).renderer=arcgis.map.renderers.ClassBreaksRenderer(class_break_infos=[cbi, cbi2], field='count', min_value=0, default_symbol=symbols.SimpleFillSymbolEsriSFS(color=[170, 170, 170, 255], outline=symbols.SimpleLineSymbolEsriSLS(color=[153, 153, 153, 64], marker=None, style='esriSLSSolid', type='esriSLS', width=0.375), style='esriSFSSolid', type='esriSFS'))


    # Attach the handler
    number_box.observe(on_number_submit, names="value")
    
    # Display widgets
    return number_box


In [28]:

# Call full pipeline for start to finish creation of our map.
my_map = create_map()
add_layer(my_map, df, df2)
combobox, output = create_combobox(my_map, crime_df, shape_df)
pointsbox = create_pointsbox(my_map)
opacitybox = create_opacitybox(my_map)
numberbox = create_numberbox(my_map)
widgets.VBox([widgets.HBox([combobox, output, pointsbox, opacitybox, numberbox]), my_map])
